<a href="https://colab.research.google.com/github/anujsahani01/English-Marathi-Translation/blob/main/Data_Preprocessing(translation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install kaggle
!pip install datasets
!pip install --upgrade accelerate
!pip install wandb
!pip uninstall transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install evaluate
!pip install rouge_score
!pip install sacremoses
!pip install sacrebleu
!pip install langid

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import langid
import pandas as pd
import datasets
import statistics

In [ ]:
from datasets import load_dataset
dataset = load_dataset("anujsahani01/English-Marathi")
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['english', 'marathi'],
        num_rows: 2637962
    })
    test: Dataset({
        features: ['english', 'marathi'],
        num_rows: 879321
    })
})

In [ ]:
def filter(data):
  data = data.replace('*', '').replace(':', '').replace(';', '').replace('$', '').replace('#', '').replace('@', '').replace('%', '').replace('&', '').replace('-', '').replace('_', '').replace('+', '').replace('~', '').replace('`', '').replace('>', '').replace('<', '').replace("|", '').replace('^', '').replace('(', '').replace(')', '').replace('{', '').replace('}', '').replace('!', '').replace('"', '').replace('[', '').replace(']', '').replace('删', '').replace('=', '').replace("'", '').replace('/', '').replace("⚡", " ").replace(".", '').replace('🐰', '').replace("【", '').replace('?', '')
  return data

In [ ]:
def lang(word):
  try:
    language = langid.classify(word)[0]
  except:
    language = None
  return language

In [ ]:
df = pd.DataFrame()
#  working with half of the data as the dataset is too large
df['en'] = dataset['train']['english'][:int(len(dataset['train'])/2.5)]
df['mr'] = dataset['train']['marathi'][:int(len(dataset['train'])/2.5)]

df1 = pd.DataFrame()
df1['en'] = dataset['test']['english'][:int(len(dataset['test'])/2.5)]
df1['mr'] = dataset['test']['marathi'][:int(len(dataset['test'])/2.5)]

In [ ]:
def process(df):
  # removing special characters from english text data
  df['en_new'] = list(map(lambda x: filter(x) , df['en']))
  df['mr_new'] = list(map(lambda x: x , df['mr']))
  # drop the previous english and marathi columns
  df.drop(['en', 'mr'], axis = 1, inplace = True)
  # extracting the first 5 letters for langauge detection( masking it a bit faster)
  df['en_wrd'] = list(map(lambda x: x[:5] , df['en_new']))
  df['mr_wrd'] = list(map(lambda x: x[:5] , df['mr_new']))
  # performing language detection using langid function made above
  return df

def filter_lang(df):
  df['inp_lang'] = df['en_wrd'].map(lang)
  df['out_lang']  = df['mr_wrd'].map(lang)
  # dropping coulmns in no language is detected
  df.dropna(subset = ['inp_lang', 'out_lang'], inplace = True)
  # dropping other languages if present in the dataset
  if df['inp_lang'].value_counts()['en'] != len(df['inp_lang']):
    df.drop(df[df['inp_lang'] != 'en'].index, inplace = True)
  # Not do language detection for maratahi as it can be confused with hindi
  df.reset_index(inplace = True)
  df.drop(['en_wrd', 'mr_wrd', 'index', 'inp_lang'm 'out_lang'], axis = 1, inplace = True)
  # dropping duplicate line to make dataset more effective
  df.drop_duplicates(inplace = True)
  # finding out the length of english data
  df['en_len'] = df['en_new'].apply(lambda row: len(row.split(' ')))
  return df

In [ ]:
df_1 = process(df)
df_train = filter_lang(df_1)

In [ ]:
print(len(df_train))
df_train.head(5)

608385


,en_new,mr_new,en_len
0,Next few months are really crucial for us,पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत.,8
1,"Kanwal Singh Chauhan, the president of Sonepat...","वृत्तसंस्थेने दिलेल्या माहितीनुसार, विरोध करण्...",15
2,He has won several competitions at the state a...,राज्य व राष्ट्रीय पातळीवरील अनेक मल्ल त्यांनी ...,11
3,This is a worrying increase,ही वाढ चिंताजनक तर आहेच.,5
4,Most of the victims were Dalits,त्यातील बहुतांशी घटनांत पीडित व्यक्ती या दलित ...,6


In [ ]:
df_2 = process(df1)
df_test = filter_lang(df_2)

In [ ]:
df_test.head(4)

,en_new,mr_new,en_len
0,This name is derived from two Sanskrit words K...,हे नाव दोन संस्कृत शब्दांकडून प्राप्त झाले आहे...,11
1,"Speaking on the occasion, Minister of State of...","रेल्वे राज्यमंत्री सुरेश अंगडी, भारतीय उद्योग ...",21
2,Two accused of raping minor girl arrested,अल्पवयीन मुलीवर अत्याचार करणाऱ्या दोन नराधमांन...,7
3,There was a man of the hill country of Ephraim...,एफ्राइमच्या डोंगराळ प्रदेशात मीखा नामक एक माणू...,14


In [ ]:
len(df_test)

327569

In [ ]:
# combining the two datasets
frames  = [df_train, df_test]
df_final = pd.concat(frames)
df_final.head(10)

,en_new,mr_new,en_len
0,Next few months are really crucial for us,पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत.,8
1,"Kanwal Singh Chauhan, the president of Sonepat...","वृत्तसंस्थेने दिलेल्या माहितीनुसार, विरोध करण्...",15
2,He has won several competitions at the state a...,राज्य व राष्ट्रीय पातळीवरील अनेक मल्ल त्यांनी ...,11
3,This is a worrying increase,ही वाढ चिंताजनक तर आहेच.,5
4,Most of the victims were Dalits,त्यातील बहुतांशी घटनांत पीडित व्यक्ती या दलित ...,6
5,This has shocked the Congress,त्यामुळे काँग्रेसला मोठा धक्का बसला.,5
6,The funds collected are used for social purposes,त्या निधीचा वापर सामाजिक कार्यासाठी करतो.,8
7,"Similarly, there is a distance restriction for...",त्याचप्रमाणे रॅक्स साठी लोडींग पॉईंट्स म्हणजे ...,34
8,"She said, I needed to have my counsellor on th...",'चित्रपटाच्या सेटवर मला माझ्या समुपदेशकांची गर...,16
9,Dont remember,"आठवले, ना.",2


In [ ]:
len(df_final)

935954

In [ ]:
print('Minimum:', df_final['en_len'].min(), 'Maximum: ' , df_final['en_len'].max(), 'Mean: ', df_final['en_len'].mean(), 'Mode:' , statistics.mode(df_final['en_len']))

Minimum: 1 Maximum:  2411 Mean:  10.481850603768988 Mode: 5


In [ ]:
# so we will be lossing this many data points
count = 0
for i in range(4):
  if i in list(df_final['en_len']):
    count = count + df_final['en_len'].value_counts()[i]

print(count)

81673


In [ ]:
df_final.drop(df_final[df_final['en_len'] <= 4].index , inplace =True, axis = 0)
df_final.reset_index(inplace = True)
df_final.drop(['index', 'en_len'], axis = 1, inplace = True)

In [ ]:
len(df_final)

671781

Observing the length after filtering the sentences

In [ ]:
df_final['en_len'] = df_final['en_new'].apply(lambda row: len(row.split(' ')))
print('Minimum:', df_final['en_len'].min(), 'Maximum: ' , df_final['en_len'].max(), 'Mean: ', df_final['en_len'].mean(), 'Mode:' , statistics.mode(df_final['en_len']))
df_final.drop(['en_len'], axis = 1, inplace = True)

Minimum: 5 Maximum:  2411 Mean:  12.03523767418251 Mode: 5


In [ ]:
split = int(0.80*len(df_final))

df_train = df_final[:split]
df_test = df_final[split:]

In [ ]:
df_train = df_train.sample(frac = 1)
df_train.reset_index(inplace = True)
df_test = df_test.sample(frac = 1)
df_test.reset_index(inplace= True)

In [ ]:
df_train.drop(['index'], inplace = True, axis = 1)
df_test.drop(['index'], inplace = True, axis = 1)

In [7]:
df_train.to_csv('train.csv' , index = False , encoding = 'utf-8')
df_test.to_csv('test.csv' , index = False , encoding = 'utf-8')